# Week 5 Notes

## 5.1 [Intro / Session overview](github.com/kemaldahha/machine-learning-course/blob/main/01-intro.md)

This week we will take the churn prediction model from week 4 that we have inside a Jupyter notebook and deploy it. We will save the model and use it.

We have the Jupter notebook with the model. The model is saved. We will create a churn service with the model. The model can be served in a web service and interacted with. 

The churn prediction model will be saved and be put inside a web service using Flask (framework for creating a web framework in Python). We will isolate the dependencies for this web service so they do not interfere with other services on our machine. To this end we'll create a special environment for Python dependencies using Pipenv. Then we add another layer on top with system dependencies, using Docker. This is then deployed in the cloud (AWS Elastic Beanstalk).

<img src=architecture.png width=600>


## 5.2 [Saving and loading the model](github.com/kemaldahha/machine-learning-course/blob/main/02-pickle.md)


Here is the code from the last week. Right now, it lives in this Jupyter Notebook and we cannot put it in a web service.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv("data-week-3.csv")

df.columns = df.columns.str.lower().str.replace(" ", "_")

categorical_columns = list(df.dtypes[df.dtypes == "object"].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(" ", "_")

df.totalcharges = pd.to_numeric(df.totalcharges, errors="coerce")
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == "yes").astype(int)

In [3]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [4]:
numerical = ["tenure", "monthlycharges", "totalcharges"]

In [6]:
categorical = [
    'gender',
    "seniorcitizen",
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [16]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient="records")
    
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)
    
    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)

    return dv, model

In [17]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient="records")

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [18]:
C = 1.0
n_splits = 5

In [19]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print(f"C={C} {np.mean(scores):.3f} += {np.std(scores):.3f}")


/home/kd/mambaforge/envs/ml-zoomcamp-py313/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/kd/mambaforge/envs/ml-zoomcamp-py313/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

C=1.0 0.842 += 0.007


/home/kd/mambaforge/envs/ml-zoomcamp-py313/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
scores

[np.float64(0.8446632807655171),
 np.float64(0.8452295225797907),
 np.float64(0.833257074051776),
 np.float64(0.8346889588795804),
 np.float64(0.8517617897147877)]

We will save the model using `pickle`:

In [22]:
import pickle

In [24]:
output_file = f"model_C={C}.bin"

In [26]:
f_out = open(output_file, "wb")
pickle.dump((dv, model), f_out)
f_out.close()

In [28]:
with open(output_file, "wb") as f_out:
    pickle.dump((dv, model), f_out)

This is how we can load the model

In [43]:
input_file = "model_C=1.0.bin"

In [44]:
with open(output_file, "rb") as f_in:
    dv, model = pickle.load(f_in)

In [45]:
model

LogisticRegression(max_iter=1000)

Now let's say we have this customer:

In [61]:
customer = {
    'gender': 'male',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'yes',
    'phoneservice': 'yes',
    'multiplelines': 'no',
    'internetservice': 'no',
    'onlinesecurity': 'no_internet_service',
    'onlinebackup': 'no_internet_service',
    'deviceprotection': 'no_internet_service',
    'techsupport': 'no_internet_service',
    'streamingtv': 'no_internet_service',
    'streamingmovies': 'no_internet_service',
    'contract': 'two_year',
    'paperlessbilling': 'no',
    'paymentmethod': 'mailed_check',
    'tenure': 12,
    'monthlycharges': 19.7,
    'totalcharges': 258.35
 }

We can predict churn as follows:

In [62]:
X = dv.transform([customer])
model.predict_proba(X)[0, 1]


np.float64(0.02597332043593159)

So now we can save a model, load it, use it. But we want to have it in a separate python file. We can create [train.py](train.py) and [predict.py](predict.py) and run them with Python. Next we will create a web service which uses these files.


## 5.3 [Web services: introduction to Flask](github.com/kemaldahha/machine-learning-course/blob/main/03-flask-intro.md)


Flask is a Python framework for creating web services. We want to encapsulate our model inside a web service called 'churn service'. 

Web service is a method for 2 devices to communicate over a network. Some information is sent to the web service with a request, then information is sent back by the web service. So we will send information on a customer and the web service will return a churn prediction.

We can create a file called `ping.py`:

```python
from flask import Flask

app = Flask("ping")

@app.route("/ping", methods=["GET"])
def ping():
    return "PONG"

if __name__=="__main__":
    app.run(debug=True, host="0.0.0.0", port="9696")
```

If we run this file, we can go to the browser or cmd with curl and type `127.0.0.1:9696/ping` or `localhost:9696/ping` and it will return `"PONG"`. This is a minimal example of creating a web service using Flask.

We can also use FastAPI as follows:
```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/ping")
async def ping():
    return "PONG"

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=9696)
```


## 5.4 [Serving the churn model with Flask](github.com/kemaldahha/machine-learning-course/blob/main/04-flask-deployment.md)


This lesson we will take the `predict.py` script and turn it into a web service using Flask. This script is saved here: [week_5/predict.py](predict.py). I have commented the code. If we run `predict.py`, a web server will be started and we can send a post request to it:

In [84]:
import requests

url = "http://localhost:9696/predict"

requests.post(url, json=customer).json()

{'churn': False, 'churn_probability': 0.03231290082794979}

This runs a Flask development server. Although this makes it easier to debug by allowing for automatic reload and providing error messages, it is not scalable, performant, and secure. Therefore we need a production server instead (e.g. `gunicorn`). We can pip install it and then run it as follows from the command line:
```bash
gunicorn --bind 0.0.0.0:9696 predict:app
```


## 5.5 [Python virtual environment: Pipenv](github.com/kemaldahha/machine-learning-course/blob/main/05-pipenv.md)


If we have multiple services running side-by-side, they may require different versions of a library. This is not possible within one environment. To address this, we can use `pipenv`, which makes use of virtual environments. These are isolated environments with their own version of pip and python.

There are different ways of managing virtual environments:
- `virtualenv` / `venv` (recommended)
- `conda`
- `pipenv`
- `poetry`

We will use pipenv in this lesson.

In [82]:
!pip install pipenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 5.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 5.7 MB/s eta 0:00:00a 0:00:01


In the command line we can use pipenv to install the packages we need:
```bash
pipenv install numpy scikit-learn==1.5.2 flask
```

This creates a Pipfile and Pipfile.lock. The Pipfile contains an overview of the packages and the versions (similar to requirements.txt) and Pipfile.lock gives an overview of all dependencies and sub-dependencies, making the environment exactly reproducible. If we install gunicorn now, the files are updates:
```bash
pipenv install gunicorn
```

Let's say this repo is cloned to another computer later. The only thing to do is ```pipenv install``` and all the dependencies will be installed.

If we type `pipenv shell` in the command line, it will start a shell for the virtual environment we created. Then we can run the command:
```bash
gunicorn --bind 0.0.0.0:9696 predict:app
```

Alternatively, we can run:
```bash
pipenv run gunicorn --bind 0.0.0.0:9696 predict:app
```

## 5.6 [Environment management: Docker](github.com/kemaldahha/machine-learning-course/blob/main/06-docker.md)


### Why we need Docker

Virtual environments can isolate dependencies. But Docker can isolate an application entirely from the rest of the computer using containers. Each container can use different versions of everything, not just python dependencies.

<img src=docker.png width=600>

It's also easy to deploy Docker containers to the cloud.

### How to run Docker

We can run docker in interactive mode my:

```bash
docker run --platform linux/amd64 -it --rm python:3.8.12-slim
```

Let's break it down:
- `3.8.12-slim` is a tag. We can find vaious Docker images in Docker Hub.
- `--platform linux/amd64` is to specify the platform. It wasn't working otherwise.
- `-it` indicates we want it in interactive mode, such that we can play around with it (otherwise it immediately closes).
- `--rm` indicates we want to get rid of the image afterwards.

This starts the container and runs python. We can change the default entrypoint:
```bash
docker run --platform linux/amd64 -it --rm --entrypoint=bash python:3.8.12-slim
```

This runs the bash shell and we can then install things. We can create a Dockerfile to do this in an automated way:

```Dockerfile
FROM python:3.13-slim

RUN pip install pipenv

WORKDIR /app
COPY ["Pipfile", "Pipfile.lock", "./"]
```

Let's break this down:
- `FROM` indicates the base image we want to use
- `RUN` can run commands, such as pip install
- `WORKDIR` will create a directory and cd into it
- `COPY` will copy files to the directory at the end of the list (`"./"` is the current directory)

We can build the Docker image using with the Dockerfile using:

```bash
docker build -t zoomcamp-test .
```

`-t zoomcamp-test` is to specify the tag
`.` is to run the Dockerfile from the current directory

Then we can run the docker image:

```bash
docker run -it --rm --entrypoint=bash zoomcamp-test
```

We will be in the directory `app` and the files we copies over will be in there:

```bash
root@6a94c5952cae:/app# ls
Pipfile  Pipfile.lock
```

We can install all dependencies now with `pipenv install`.

We don't need to manually run pipenv install. We can update the Dockerfile:

```Dockerfile
FROM python:3.13-slim

RUN pip install pipenv

WORKDIR /app
COPY ["Pipfile", "Pipfile.lock", "./"]

RUN pipenv install
```

However, this will create a virtual environment inside the Docker image, which is already isolated. We just want the dependencies installed (why not use `requirements.txt` then though?). So we can update the Dockerfile with `--system --deploy`:

```Dockerfile
FROM python:3.13-slim

RUN pip install pipenv

WORKDIR /app
COPY ["Pipfile", "Pipfile.lock", "./"]

RUN pipenv install --system --deploy
```

Next, let's copy `predict.py` and the model:

```Dockerfile
FROM python:3.13-slim

RUN pip install pipenv

WORKDIR /app
COPY ["Pipfile", "Pipfile.lock", "./"]

RUN pipenv install --system --deploy

COPY ["predict.py", "model_C=1.0.bin", "./"]
```

We can build the Dockerfile again and run the image. Then we run gunicorn:

```bash
gunicorn --bind=0.0.0.0:9696 predict:app
```

However, we cannot access it yet. We need to expose port 9696 of the container and map it to a port on the host machine. Also we change the entrypoint to run the `predict.py` app with `gunicorn`: 

```Dockerfile
FROM python:3.13-slim

RUN pip install pipenv

WORKDIR /app
COPY ["Pipfile", "Pipfile.lock", "./"]

RUN pipenv install --system --deploy

COPY ["predict.py", "model_C=1.0.bin", "./"]

EXPOSE 9696

ENTRYPOINT ["gunicorn", "--bind=0.0.0.0:9696", "predict:app"]
```

Finally, when running Docker, we need to map the ports as follows:

```bash
docker run -it --rm -p 9696:9696 zoomcamp-test
```

Now we can run `predict-test.py` or `curl` and get the prediction from `predict.py` as a web service from a Docker container.

The next step is to deploy this container to the cloud, which is what we'll do in Lesson 5.7.


## 5.7 [Deployment to the cloud: AWS Elastic Beanstalk (optional)](github.com/kemaldahha/machine-learning-course/blob/main/07-aws-eb.md)


Now we want to deploy the Docker image to the cloud. We will use AWS Elastic Beanstalk.

Elastic Beanstalk can automatically scale up an down as well as balance load.

We need to install awsebcli. In the folder with our `predict.py` and `train.py` files, we can run:

```bash
pipenv install awsebcli --dev
```

`--dev` ensures it's designated as only to be used for development, not production.

Then we go to the shell:

```bash
pipenv shell
eb
```

Now we initialize the service:

```bash
eb init -p docker -r eu-west-1 churn-serving
```

- `-p docker` specifies the platform (in this case, Docker)
- `-r eu-west-1` specifies the region to deploy the application
- `churn-serving` specifies the name of the Elastic Beanstalk application.

This should create a folder called `.elasticbeanstalk`

We can check locally with EB if it works:

```bash
eb local run --port 9696
```

Note that initially I got this error:

```bash
ERROR: NotSupportedError - You can use "eb local" only with preconfigured, generic and multicontainer Docker platforms.
```

I resolved it by following a solution in the FAQ:

```bash
eb init -i
```

And then selecing: `Docker running on 64bit Amazon Linux 2023`

After doing this, the `eb local run --port 9696` runs OK and the churn service is up.

Now that it's confirmed to work, we will deploy the application to the cloud:

```bash
eb create churn-serving-env
```

I got this error:

```bash
2024-10-30 06:16:10    ERROR   Creating Auto Scaling launch configuration failed Reason: Resource handler returned message: "The Launch Configuration creation operation is not available in your account. Use launch templates to create configuration templates for your Auto Scaling groups. (Service: AutoScaling, Status Code: 400, Request ID: c47ea712-2979-46bd-b44c-59362b37f48e)" (RequestToken: d75111ee-1af1-d748-ed7d-c4a4de863b56, HandlerErrorCode: GeneralServiceException)
```

This one is also covered in the FAQ. The solution is to just run `eb create` and selecting `classic` load balancer type. See details in FAQ.

Now the application is in the cloud. In `predict-test.py` we can adjust the post request by replacing `localhost:9696` with the host from the EB environment and use the web service.

Finally, we terminate the web service:

```bash
eb terminate churn-serving-env
```


## 5.8 [Summary](github.com/kemaldahha/machine-learning-course/blob/main/08-summary.md)


No notes.


## 5.9 [Explore more](github.com/kemaldahha/machine-learning-course/blob/main/09-explore-more.md)


No notes.


## 5.10 [Homework](github.com/kemaldahha/machine-learning-course/blob/main/homework.md)

Go to [week_5_notes.ipynb](week_5_notes.ipynb)